# EDA
- 결론: 한 영상 묶음은 하나의 라벨로 묶여있다.

In [1]:
import os
import glob
import pickle
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd


In [2]:
base_path = '/home/hyun/focussu-ai/data/109.학습태도_및_성향_관찰_데이터/3.개방데이터/1.데이터/Training'

metadata_list = ['00_01', '00_02', '00_03', '00_04', '00_05',
                '10_01', '10_02', '10_03',
                ]

meta_dirs = [f'{base_path}/02.라벨링데이터/TL_{base}' for base in metadata_list]



In [4]:
meta_list = []
meta_dir = base_path + '/02.라벨링데이터/*/*.json'

for meta_dir in meta_dirs:
    if not os.path.exists(meta_dir):
        print(f"경고: 디렉토리가 존재하지 않습니다: {meta_dir}")
        continue
    path_list = glob.glob(meta_dir+'/*.json')
    for path in path_list:
        with open(path, 'r') as f:
                    metadata = json.load(f)
                    meta_dict = {
                            'filename': metadata['이미지']['filename'],
                            'idx': metadata['이미지']['timeline']['id'],
                            'category_id': metadata['이미지']['category']['id'],
                            'category_name': metadata['이미지']['category']['name'],
                            'emotion': metadata['이미지']['emotion']
                    }
                    meta_list.append(meta_dict)
print(len(meta_list))

333949


In [3]:
meta_list = []
meta_dir = base_path + '/02.라벨링데이터/*/*.json'


path_list = glob.glob(meta_dir)
print(len(path_list))


1166835


In [4]:
for path in path_list:
    with open(path, 'r') as f:
                metadata = json.load(f)
                meta_dict = {
                        'filename': metadata['이미지']['filename'],
                        'idx': metadata['이미지']['timeline']['id'],
                        'category_id': metadata['이미지']['category']['id'],
                        'category_name': metadata['이미지']['category']['name'],
                        'emotion': metadata['이미지']['emotion']
                }
                meta_list.append(meta_dict)
print(len(meta_list))


1166835


In [5]:
df = pd.DataFrame(meta_list)
df.sort_values(by=['filename', 'idx'], inplace=True)

In [6]:
print(df.shape)

(1166835, 5)


In [ ]:
for k, v in df.groupby(by=['filename']):
    if(v['category_id'].nunique() > 1):
        

In [10]:
df.head()

,filename,idx,category_id,category_name,emotion
0,04-01-378-01-1-8-23082700000337-01.mp4,38,1,집중,흥미로움
1,04-01-423-02-2-9-23083100000020-01.mp4,38,1,집중,흥미로움
2,04-01-497-01-2-8-23083000000036-01.mp4,11,1,집중,흥미로움
3,04-01-544-01-1-8-23090100000009-01.mp4,17,1,집중,흥미로움
4,04-01-179-02-2-9-23082700000331-01.mp4,24,1,집중,흥미로움
